# Unsupervised topic modelling on Amazon product descriptions
This notebook creates a Latent Dirichlet Allocation model to analyse the latent topics within a dataset of Amazon products and product descriptions. 

## Setting up and preparing key functions
Adding additional stopwords to the basic English configuration, namely 'from', 'product', 'item', 'about', and 'use'.

In [ ]:
import gensim
from gensim.utils import simple_preprocess
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'product', 'item', 'about', 'use'])

def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
        
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


## Checking the dataset as DataFrame

In [ ]:
import pandas as pd
products = pd.read_csv('productnames.csv')
products.head(10)

,product_description
0,\n About this item 2 year shelf life. Only...
1,\n About this item \n This fits your .\n ...
2,\n About this item 2 year shelf life. Only...
3,\n About this item CLOROX BLEACH: Use Clor...
4,\n About this item \n This fits your .\n ...
5,"\n About this item Current fresh stock, Ex..."
6,\n About this item EFFICIENTLY BLEACH: Con...
7,\n About this item EFFICIENTLY BLEACH: Con...
8,\n About this item ULTRA CLEAN TOILET BOWL...
9,\n About this item EFFICIENTLY BLEACH: Con...


## Preparing the corpus and removing stopwords

In [ ]:
data = products.values.tolist()
data_words = list(sent_to_words(data))
# remove stop words
data_words = remove_stopwords(data_words)
print(data_words[:1][0][:30])

['year', 'shelf', 'life', 'activates', 'hot', 'cold', 'water', 'spills', 'splash', 'mess', 'one', 'tablet', 'mixed', 'cup', 'water', 'equal', 'cup', 'standard', 'regular', 'liquid', 'bleach', 'light', 'weight', 'easy', 'store', 'transport', 'less', 'plastic', 'waste', 'helps']


In [ ]:
import gensim.corpora as corpora
# Create Dictionary
id2word = corpora.Dictionary(data_words)
# Create Corpus
texts = data_words
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# View
print(corpus[:1][0][:30])

[(0, 1), (1, 2), (2, 1), (3, 2), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 2), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 2), (29, 1)]


## Building the initial LDA model

In [ ]:
from pprint import pprint
# number of topics
num_topics = 7
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.019*"laundry" + 0.014*"stains" + 0.014*"detergent" + 0.011*"safe" + '
  '0.010*"stain" + 0.010*"clean" + 0.009*"water" + 0.009*"machines" + '
  '0.008*"coffee" + 0.007*"remover"'),
 (1,
  '0.009*"moisture" + 0.009*"free" + 0.008*"clean" + 0.007*"water" + '
  '0.007*"spray" + 0.007*"cleaner" + 0.006*"xa" + 0.006*"cleaning" + '
  '0.006*"made" + 0.005*"laundry"'),
 (2,
  '0.017*"laundry" + 0.011*"detergent" + 0.011*"free" + 0.011*"cleaning" + '
  '0.009*"clean" + 0.007*"fits" + 0.007*"safe" + 0.007*"washing" + '
  '0.007*"water" + 0.006*"sheets"'),
 (3,
  '0.012*"cleaning" + 0.011*"clean" + 0.009*"laundry" + 0.008*"free" + '
  '0.008*"cleaner" + 0.007*"water" + 0.007*"safe" + 0.007*"fits" + '
  '0.007*"detergent" + 0.005*"made"'),
 (4,
  '0.011*"free" + 0.010*"laundry" + 0.010*"stains" + 0.010*"clean" + '
  '0.010*"detergent" + 0.009*"stain" + 0.008*"clothes" + 0.007*"safe" + '
  '0.007*"remover" + 0.007*"cleaning"'),
 (5,
  '0.014*"laundry" + 0.011*"detergent" + 0.010*"moistur

## Importing additional packages and libraries
PyLDAvis and os are needed for visualisation of relative intertopic distance. 

In [ ]:
!pip install pyldavis
import pyLDAvis.gensim
import pickle 
import pyLDAvis
import os

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 60.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 KB 32.1 MB/s eta 0:00:00
  Attempting uninstall: joblib
    Found existing installation: joblib 1.1.1
    Uninstalling joblib-1.1.1:
      Successfully uninstalled joblib-1.1.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pandas-profiling 3.2.0 requires joblib~=1.1.0, but you have joblib 1.2.0 which is incompatible.


## Preparing initial visualisation of intertopic distance

In [ ]:
# Visualize the topics
pyLDAvis.enable_notebook()
LDAvis_data_filepath = os.path.join('data_words10'+str(num_topics))

if 1 == 1:
    LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)
pyLDAvis.save_html(LDAvis_prepared, 'data_words10'+ str(num_topics) +'.html') # loading data_words set instead of 'data from disk'
LDAvis_prepared

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.9/dist-packages/pyLDAvis/_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.060323 -0.014638       1        1  28.095266
6     -0.058760 -0.028747       2        1  19.782078
3      0.000081 -0.005702       3        1  13.412119
4      0.026493 -0.005799       4        1  11.184289
2     -0.002213 -0.011303       5        1   9.985569
5     -0.002698  0.041907       6        1   9.688852
1     -0.023225  0.024283       7        1   7.851827, topic_info=             Term         Freq        Total Category  logprob  loglift
1927     moisture   296.000000   296.000000  Default  30.0000  30.0000
94        cleaner   749.000000   749.000000  Default  29.0000  29.0000
1012        glass   464.000000   464.000000  Default  28.0000  28.0000
65        laundry  1103.000000  1103.000000  Default  27.0000  27.0000
140        toilet   278.000000   278.000000  Default  26.0000  26.0000
...           ...          ...          ...      ...      ...      ...
382     detergent    31.208804   854.948743   Topic7  -5.4379  -0.7659
175   ingredients    24.624343   328.086842   Topic7  -5.6749  -0.0451
73         stains    28.428543   762.961381   Topic7  -5.5312  -0.7454
205          safe    26.132655   703.874276   Topic7  -5.6155  -0.7490
136         stain    23.168725   508.320572   Topic7  -5.7358  -0.5439

[611 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
3110      6  0.565089  absorbed
2644      1  0.121517  absorber
2644      2  0.194427  absorber
2644      3  0.097214  absorber
2644      4  0.072910  absorber
...     ...       ...       ...
39        3  0.143142        xa
39        4  0.054530        xa
39        5  0.143142        xa
39        6  0.177223        xa
39        7  0.153366        xa

[1740 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 7, 4, 5, 3, 6, 2])

In [ ]:
from gensim.models.coherencemodel import CoherenceModel
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_words, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)

## Baseline coherence score (stopwords, 7 topics) = 0.343113

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Coherence Score:  0.34311352054621375


In [ ]:
def compute_coherence_values(corpus, dictionary, k, a, b):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=k, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=a,
                                           eta=b)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_words, dictionary=id2word, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
import numpy as np
import tqdm

grid = {}
grid['Validation_Set'] = {}

# Topics range
min_topics = 2
max_topics = 11
step_size = 1
topics_range = range(min_topics, max_topics, step_size)

# Alpha parameter
alpha = list(np.arange(0.01, 1, 0.3))
alpha.append('symmetric')
alpha.append('asymmetric')

# Beta parameter
beta = list(np.arange(0.01, 1, 0.3))
beta.append('symmetric')

# Validation sets
num_of_docs = len(corpus)
corpus_sets = [gensim.utils.ClippedCorpus(corpus, int(num_of_docs*0.75)), 
               corpus]

corpus_title = ['75% Corpus', '100% Corpus']

model_results = {'Validation_Set': [],
                 'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }

# Can take a long time to run
if 1 == 1:
    pbar = tqdm.tqdm(total=(len(beta)*len(alpha)*len(topics_range)*len(corpus_title)))
    
    # iterate through validation corpuses
    for i in range(len(corpus_sets)):
        # iterate through number of topics
        for k in topics_range:
            # iterate through alpha values
            for a in alpha:
                # iterare through beta values
                for b in beta:
                    # get the coherence score for the given parameters
                    cv = compute_coherence_values(corpus=corpus_sets[i], dictionary=id2word, 
                                                  k=k, a=a, b=b)
                    # Save the model results
                    model_results['Validation_Set'].append(corpus_title[i])
                    model_results['Topics'].append(k)
                    model_results['Alpha'].append(a)
                    model_results['Beta'].append(b)
                    model_results['Coherence'].append(cv)
                    
                    pbar.update(1)
    pd.DataFrame(model_results).to_csv('/content/sample_data/results.csv', index=False)
    pbar.close()


 12%|█▏        | 66/540 [1:37:25<11:39:42, 88.57s/it]

100%|██████████| 540/540 [1:45:03<00:00, 11.67s/it]


In [ ]:
results = pd.read_csv('results.csv')
results.head(10)
# checking the file with results of model configurations

,Validation_Set,Topics,Alpha,Beta,Coherence
0,75% Corpus,2,0.01,0.01,0.359470
1,75% Corpus,2,0.01,0.31,0.356286
2,75% Corpus,2,0.01,0.61,0.389983
3,75% Corpus,2,0.01,0.9099999999999999,0.383450
4,75% Corpus,2,0.01,symmetric,0.382419
5,75% Corpus,2,0.31,0.01,0.376460
6,75% Corpus,2,0.31,0.31,0.359567
7,75% Corpus,2,0.31,0.61,0.382419
8,75% Corpus,2,0.31,0.9099999999999999,0.401974
9,75% Corpus,2,0.31,symmetric,0.382419


## Notes detailing the optimal parameters
Maximising the models for coherence. 

In [ ]:
## graphing coherence scores over numbers of topics
    ## K = 7
    ## alpha = 0.31
    ## beta = 0.91
    ## coherence = 59.0414

## Tayloring the model to optimal parameters

In [ ]:
num_topics = 7

lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=num_topics, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=0.31,
                                           eta=0.91)

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_words, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)
## improved coherence score = 0.588189755

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Coherence Score:  0.5881897558818407


## Visualising the final model and its intertopic distance
The last cell saves the final model. 

In [ ]:
pyLDAvis.enable_notebook()
LDAvis_data_filepath = os.path.join('data_words10'+str(num_topics))

if 1 == 1:
    LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)

with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)
pyLDAvis.save_html(LDAvis_prepared, 'data_words10'+ str(num_topics) +'.html') # loading data_words set
LDAvis_prepared

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.9/dist-packages/pyLDAvis/_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.237074  0.047328       1        1  36.309735
6     -0.012184 -0.127395       2        1  26.018684
0     -0.031736 -0.060012       3        1  17.141885
4      0.075119 -0.000789       4        1   8.562252
3      0.067474  0.015838       5        1   4.744304
5      0.041088  0.096274       6        1   3.926597
1      0.097314  0.028757       7        1   3.296543, topic_info=            Term         Freq        Total Category  logprob  loglift
65       laundry  1129.000000  1129.000000  Default  30.0000  30.0000
382    detergent   848.000000   848.000000  Default  29.0000  29.0000
136        stain   472.000000   472.000000  Default  28.0000  28.0000
73        stains   698.000000   698.000000  Default  27.0000  27.0000
1012       glass   373.000000   373.000000  Default  26.0000  26.0000
...          ...          ...          ...      ...      ...      ...
778          epa     6.825139    40.392188   Topic7  -6.0902   1.6343
1099         try     5.706710    26.958967   Topic7  -6.2691   1.8596
913      looking     5.006240    35.398205   Topic7  -6.4001   1.4563
78          used     4.716011   156.313403   Topic7  -6.4598  -0.0886
284   eliminates     4.316883   103.994335   Topic7  -6.5482   0.2305

[405 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
2825      1  0.062580  absorb
2825      2  0.062580  absorb
2825      3  0.062580  absorb
2825      4  0.750958  absorb
2825      6  0.062580  absorb
...     ...       ...     ...
5425      1  0.146961      𝐍𝐨
5425      2  0.073481      𝐍𝐨
5425      3  0.073481      𝐍𝐨
5425      4  0.073481      𝐍𝐨
5425      6  0.587844      𝐍𝐨

[1753 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 7, 1, 5, 4, 6, 2])

In [ ]:
save(lda_model)

'' was not found in history, as a file, url, nor in the user namespace.


/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
